In [74]:
import numpy as np
import scipy.constants as c
from numpy.linalg import norm

in_to_m = 25.4e-3

In [71]:
def normalize(vector):
    return vector / np.sqrt(np.sum(vector)**2)

class WireSegment:
    def __init__(self, wire_position, wire_length, current_direction):
        self._r0 = np.array(wire_position)
        self._l = wire_length
        self._l_hat = normalize(np.array(current_direction))

    def magnetic_field(self, current, position):
        displacement = np.array(position) - self._r0
        return c.mu_0 / (4 * np.pi) * current * self._l * np.cross(self._l_hat, displacement) / norm(displacement)**3

class SquareCoil:
    def __init__(self, l, cw_looking_from_neg_z = True, segments_per_side = 100):
        self._l = l
        self._cw_looking_from_neg_z = cw_looking_from_neg_z
        self._segmentize(segments_per_side)

    def _segmentize(self, segments_per_side):
        self._segments = []
        for kk in range(segments_per_side):
            wire_length = self._l / segments_per_side
            wire_position = self._l / segments_per_side * kk - self._l / 2
            for ll in [0, 1]:
                for mm in [-1, 1]:
                    position = [self._l / 2 * mm, self._l / 2 * mm, 0]
                    position[ll] = wire_position
                    if ll == 0:
                        if mm == 1:
                            current = [-1, 0, 0]
                        else:
                            current = [1, 0, 0]
                    else:
                        if mm == 1:
                            current = [0, 1, 0]
                        else:
                            current = [0, -1, 0]
                    self._segments.append(
                        WireSegment(position, wire_length, current)
                    )

    def magnetic_field(self, current, position):
        total_field = np.zeros(3, dtype=float)
        for segment in self._segments:
            total_field += segment.magnetic_field(current, position)
        return total_field

In [83]:
coil = SquareCoil(26 * in_to_m)
coil.magnetic_field(50, [0 * in_to_m, 0 * in_to_m, 10 * in_to_m])

array([0.00000000e+00, 0.00000000e+00, 4.72730205e-05])